In [1]:
import numpy as np

np.sin(3)

0.1411200080598672

In [2]:
from azureml.opendatasets import Diabetes

diabetes = Diabetes.get_tabular_dataset()
X = diabetes.drop_columns("Y")
y = diabetes.keep_columns("Y")
X_df = X.to_pandas_dataframe()
y_df = y.to_pandas_dataframe()
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AGE     442 non-null    int64  
 1   SEX     442 non-null    int64  
 2   BMI     442 non-null    float64
 3   BP      442 non-null    float64
 4   S1      442 non-null    int64  
 5   S2      442 non-null    float64
 6   S3      442 non-null    float64
 7   S4      442 non-null    float64
 8   S5      442 non-null    float64
 9   S6      442 non-null    int64  
dtypes: float64(6), int64(4)
memory usage: 34.7 KB


In [5]:
# 糖尿病指數 並非是糖尿病 vs 正常(分類)
X_df[:3], y_df[:3]

(   AGE  SEX   BMI     BP   S1     S2    S3   S4      S5  S6
 0   59    2  32.1  101.0  157   93.2  38.0  4.0  4.8598  87
 1   48    1  21.6   87.0  183  103.2  70.0  3.0  3.8918  69
 2   72    2  30.5   93.0  156   93.6  41.0  4.0  4.6728  85,
      Y
 0  151
 1   75
 2  141)

In [6]:
import joblib
from sklearn.linear_model import Ridge

model = Ridge().fit(X_df,y_df)
joblib.dump(model, 'sklearn_regression_model.pkl') # 輸出到哪裡?!

['sklearn_regression_model.pkl']

In [7]:
# 將上述資料進行註冊
import sklearn

from azureml.core import Workspace
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

ws = Workspace.from_config()

model = Model.register(workspace=ws,
                       model_name='my-sklearn-model',                # Name of the registered model in your workspace.
                       model_path='./sklearn_regression_model.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version=sklearn.__version__,  # Version of scikit-learn used to create the model.
                       sample_input_dataset=X,
                       sample_output_dataset=y,
                       resource_configuration=ResourceConfiguration(cpu=2, memory_in_gb=4),
                       description='Ridge regression model to predict diabetes progression.',
                       tags={'area': 'diabetes', 'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

Registering model my-sklearn-model
Name: my-sklearn-model
Version: 1


In [8]:
%%writefile score.py

import json
import pickle
import numpy as np
import pandas as pd
import os
import joblib
from azureml.core.model import Model

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


def init():
    global model
    # Replace filename if needed.
    path = os.getenv('AZUREML_MODEL_DIR') 
    model_path = os.path.join(path, 'sklearn_regression_model.pkl')
    # Deserialize the model file back into a sklearn model.
    model = joblib.load(model_path)


input_sample = pd.DataFrame(data=[{
    "AGE": 5,
    "SEX": 2,
    "BMI": 3.1,
    "BP": 3.1,
    "S1": 3.1,
    "S2": 3.1,
    "S3": 3.1,
    "S4": 3.1,
    "S5": 3.1,
    "S6": 3.1
}])

# This is an integer type sample. Use the data type that reflects the expected result.
output_sample = np.array([0])

# To indicate that we support a variable length of data input,
# set enforce_shape=False
@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        print("input_data....")
        print(data.columns)
        print(type(data))
        result = model.predict(data)
        print("result.....")
        print(result)
    # You can return any data type, as long as it can be serialized by JSON.
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

Writing score.py


In [9]:
#機器上面py conda 虛擬環境
from azureml.core.model import InferenceConfig
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'pandas',
    'scikit-learn=={}'.format(sklearn.__version__)
])

inference_config = InferenceConfig(entry_script='./score.py',environment=environment)

In [10]:
service_name = 'my-diabetes-model'

service = Model.deploy(ws, service_name, [model], inference_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-09-25 03:41:38+00:00 Creating Container Registry if not exists..
2022-09-25 03:51:39+00:00 Registering the environment.
2022-09-25 03:51:41+00:00 Building image..
2022-09-25 04:01:56+00:00 Generating deployment configuration.
2022-09-25 04:01:58+00:00 Submitting deployment to compute..
2022-09-25 04:02:05+00:00 Checking the status of deployment my-diabetes-model..
2022-09-25 04:04:29+00:00 Checking the status of inference endpoint my-diabetes-model.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [13]:
import json
print('service', service)
input_payload = json.dumps({
    'data': X_df[0:2].values.tolist()
})

output = service.run(input_payload)
print('-' * 50)
print('output', output)

service AciWebservice(workspace=Workspace.create(name='azml0925', subscription_id='2534c950-f9a2-46bb-b43f-fb87ead1a2f9', resource_group='eastasia'), name=my-diabetes-model, image_id=None, image_digest=None, compute_type=ACI, state=Healthy, scoring_uri=http://3beaab7a-5a12-4130-be5e-bbd72f8c25e5.eastasia.azurecontainer.io/score, tags={}, properties={'hasInferenceSchema': 'True', 'hasHttps': 'False', 'authEnabled': 'False'}, created_by={'userObjectId': '9d9b954b-6a6b-47ee-918b-55e143e01a3d', 'userPuId': '10037FFE8B9E5256', 'userIdp': None, 'userAltSecId': None, 'userIss': 'https://sts.windows.net/3e04753a-ae5b-42d4-a86d-d6f05460f9e4/', 'userTenantId': '3e04753a-ae5b-42d4-a86d-d6f05460f9e4', 'userName': 'Lewis Yang', 'upn': 'lewis_yang@trendmicro.com'})
--------------------------------------------------
output [[205.59094435613133], [68.84146418576978]]
